In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import deepmatcher as dm
import pandas as pd
import pickle
import torch
from tqdm import tqdm
from nltk.corpus import stopwords
import copy
from utils.intermediate_layer_extraction import returnLayerInputUnlabeled
from utils.intermediate_layer_extraction import return_layer_input
import torch.nn as nn

In [3]:
from utils.attacker import getAttackedDataset

In [ ]:
def canBeReplaced(perturbationL,attr,newdiff):
    for att,val in perturbationL:
        if att==attr and val < newdiff:
            return False
    return True

In [ ]:
def evaluatePerturbations(model,test_v,perturbations_v,pairAttackMap,pairPerturbedAttr):
    sampleAttackSuccessFull = {}
    for key in tqdm(test_v):
        sampleAttackSuccessFull[key] = []
        logsoftmax = nn.LogSoftmax(dim=1)
        originalPred = logsoftmax(model.classifier.forward(torch.unsqueeze(test_v[key],0)))
        if originalPred[0][0].exp().item() > originalPred[0][1].exp().item():
            originalPred = 0
        else:
            originalPred = 1
        correspondingAttacks = pairAttackMapping[key]
        for idx,attackid in enumerate(correspondingAttacks):
            perturb = perturbations_v[attackid]
            currentPred = logsoftmax(model.classifier.forward(torch.unsqueeze(perturb,0)))
            if currentPred[0][0].item() > currentPred[0][1].item():
                currentPred = 0
            else:
                currentPred = 1
            if currentPred != originalPred:
                difference = torch.norm(perturb-test_v[key]).item()
                perturbedAttribute = pairPerturbedAttr[key].iloc[idx]
                if canBeReplaced(sampleAttackSuccessFull[key],perturbedAttribute,difference):
                    sampleAttackSuccessFull[key].append((perturbedAttribute,difference))
    return sampleAttackSuccessFull

In [4]:
f1 = open('attacksResults/closest_words_itam.pickle', 'rb')
closest_words_map = pickle.load(f1)
f1.close()
f2 = open('attacksResults/notfound_words_itam.data', 'rb')
notfound = pickle.load(f2)
f2.close()

In [5]:
test_df = pd.read_csv('datasets/Structured/itunes-amazon/merged_test.csv')

In [7]:
attributes = [col for col in list(test_df) if col not in ['id','label']]
itunesamazon_model = dm.MatchingModel(attr_summarizer='hybrid')
itunesamazon_model.load_state('models/itunes_amazon_hybrid.pth')
itunesamazon_model = itunesamazon_model.to('cuda')

In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
attacked_test,pairAttackMapping,pairPerturbedAttr = getAttackedDataset(test_df,attributes,
                                                                       closest_words_map,notfound,stop_words)
attacked_test.to_csv('itunes_amazon_attacked_test.csv',index=False)

100%|██████████| 109/109 [00:12<00:00,  8.58it/s]


In [11]:
model = copy.deepcopy(itunesamazon_model)

In [12]:
test_v = returnLayerInputUnlabeled(model,model.classifier,
                             'datasets/Structured/itunes-amazon/','merged_test')


Reading and processing data from "datasets/Structured/itunes-amazon/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00

In [13]:
attackedTest_v = returnLayerInputUnlabeled(model,model.classifier,
                                   './','itunes_amazon_attacked_test',batch_size=32,
                                           ignore_columns=['id','label','altered_attribute'])


Reading and processing data from "./itunes_amazon_attacked_test.csv"
0% [##############################] 100% | ETA: 00:00:00

In [29]:
ri = evaluatePerturbations(itunesamazon_model,test_v,attackedTest_v,pairAttackMapping,pairPerturbedAttr)

100%|██████████| 109/109 [00:27<00:00,  4.01it/s]


In [30]:
ri = {k: v for k, v in ri.items() if len(v)>0}
ri

{722: [('rtable_Time', 3.3940348625183105)],
 646: [('ltable_Time', 8.449257850646973), ('rtable_Time', 8.449188232421875)],
 718: [('rtable_Time', 2.205073356628418)],
 735: [('ltable_Time', 3.1211178302764893)],
 663: [('ltable_Time', 2.511775493621826), ('rtable_Time', 3.106961727142334)],
 678: [('ltable_Time', 5.2526116371154785)],
 694: [('ltable_Price', 2.09537672996521),
  ('ltable_Price', 1.4944628477096558)],
 724: [('ltable_Song_Name', 0.9200679659843445),
  ('ltable_Album_Name', 2.584613084793091),
  ('ltable_Album_Name', 1.8332784175872803),
  ('ltable_Time', 2.423051118850708),
  ('rtable_Song_Name', 1.8313816785812378),
  ('rtable_Song_Name', 1.812230110168457)],
 696: [('ltable_Time', 6.862666130065918), ('rtable_Time', 6.861691951751709)],
 649: [('ltable_Time', 3.8935327529907227),
  ('rtable_Time', 3.8381967544555664)],
 638: [('rtable_Time', 4.896437644958496)],
 635: [('rtable_Time', 5.7664713859558105)],
 738: [('ltable_Time', 2.849313735961914)],
 664: [('ltable_

## Amazon-Google

In [38]:
test_df = pd.read_csv('datasets/Structured/Amazon-Google/merged_test.csv')
attributes = [col for col in list(test_df) if col not in ['id','label']]
amazongoogle_model = dm.MatchingModel(attr_summarizer='hybrid')
amazongoogle_model.load_state('models/amazongoogle_hybrid.pth')
amazongoogle_model = amazongoogle_model.to('cuda')

In [35]:
f1 = open('attacksResults/closest_words_google.pickle', 'rb')
closest_words_map = pickle.load(f1)
f1.close()
f2 = open('attacksResults/notfound_google.data', 'rb')
notfound = pickle.load(f2)
f2.close()

In [32]:
model = copy.deepcopy(amazongoogle_model)

In [33]:
test_v = returnLayerInputUnlabeled(model,model.classifier,
                             'datasets/Structured/Amazon-Google/','merged_test')


Reading and processing data from "datasets/Structured/Amazon-Google/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00

In [39]:
attacked_test,pairAttackMapping,pairPerturbedAttr = getAttackedDataset(test_df,attributes,
                                                                       closest_words_map,notfound,stop_words)
attacked_test.to_csv('amazongoogle_attacked_test.csv',index=False)

100%|██████████| 2293/2293 [01:22<00:00, 27.93it/s]


In [40]:
attackedTest_v = returnLayerInputUnlabeled(model,model.classifier,
                                   './','amazongoogle_attacked_test',batch_size=32,
                                           ignore_columns=['id','label','altered_attribute'])


Reading and processing data from "./amazongoogle_attacked_test.csv"
0% [##############################] 100% | ETA: 00:00:00

In [ ]:
ri = evaluatePerturbations(model,test_v,attackedTest_v,pairAttackMapping,pairPerturbedAttr)

 94%|█████████▎| 2149/2293 [02:32<00:09, 14.51it/s]

In [ ]:
ri = {k: v for k, v in ri.items() if len(v)>0}
ri